# Generate VIIRS points

**Name:** Asaf Vanunu  
**Date:** July 31, 2024

## Description
This script takes VIIRS fire product and geolocation files and create VIIRS shapefile points


* Import libraries

In [13]:
import GOES_VIIRS_tools
import os
import pandas as pd
import numpy as np

* set path

In [2]:
VIIRS_fire_product_dir = "F:\\VIIRS_data\\VIIRS_fire"
VIIRS_geolocation_dir = "F:\\VIIRS_data\\VIIRS_GEO"

* Create a file list for the fire and geolocation products

In [7]:
## Get the list of VIIRS fire and geolocation files
VIIRS_fire_files = [os.path.join(VIIRS_fire_product_dir, fire_file) for fire_file in os.listdir(VIIRS_fire_product_dir) if fire_file.endswith("nc")]
VIIRS_geolocation_files = [os.path.join(VIIRS_geolocation_dir, geo_file) for geo_file in os.listdir(VIIRS_geolocation_dir) if geo_file.endswith("nc")]
print(VIIRS_fire_files[0]) 
print(VIIRS_geolocation_files[0])

F:\VIIRS_data\VIIRS_fire\VNP14IMG.A2022182.0906.001.2022183034226.nc
F:\VIIRS_data\VIIRS_GEO\VNP03IMG.A2022182.0906.002.2022182155401.nc


- Now we need to create a matching list of VIIRS fire files to VIIRS geolocation files
- We will start by creating a function the filter matching VIIRS 

In [9]:
def get_unique_id_from_VIIRS_file(VIIRS_file):
    """This function extracts the unique ID from the VIIRS file name. This is for example A2022182.0906

    Args:
        VIIRS_file (string): The VIIRS file name
    """
    VIIRS_file_split = VIIRS_file.split("\\") ## Split the file path
    VIIRS_file_name = VIIRS_file_split[-1] ## Get the file name
    VIIRS_file_name_split = VIIRS_file_name.split(".") ## Split the file name
    VIIRS_unique_id = VIIRS_file_name_split[1] ## Get the unique ID
    VIIRS_unique_time = VIIRS_file_name_split[2] ## Get the unique time
    
    out_string = f"{VIIRS_unique_id}.{VIIRS_unique_time}" ## Create the output string
    return out_string
    

In [10]:
print(get_unique_id_from_VIIRS_file(VIIRS_fire_files[0]))
print(get_unique_id_from_VIIRS_file(VIIRS_geolocation_files[0]))

A2022182.0906
A2022182.0906


* Now we will create the matching lists

In [15]:
matching_VIIRS_fires = [] ## List to store the matching VIIRS fire files
matching_VIIRS_geolocations = [] ## List to store the matching VIIRS geolocation files
geo_files_unique_ids = list(map(get_unique_id_from_VIIRS_file, VIIRS_geolocation_files)) ## Get the unique IDs of the geolocation files

for i in range(len(VIIRS_fire_files)): ## Loop through the VIIRS fire files
    VIIRS_fire_unique_id = get_unique_id_from_VIIRS_file(VIIRS_fire_files[i]) ## Get the unique ID of the VIIRS fire file
    VIIRS_fire_file_condition = np.isin(VIIRS_fire_unique_id, geo_files_unique_ids) ## Check if the unique ID is in the list of unique IDs
    if VIIRS_fire_file_condition: ## If the unique ID is in the list of unique IDs
        matching_VIIRS_fires.append(VIIRS_fire_files[i]) ## Append the matching VIIRS fire file
        matching_VIIRS_geolocations.append(VIIRS_geolocation_files[geo_files_unique_ids.index(VIIRS_fire_unique_id)]) ## Append the matching VIIRS geolocation file

* Now we will see if the two lists are matching

In [20]:
con_list = [] ## List to store the conditions
if len(matching_VIIRS_fires) == len(matching_VIIRS_geolocations): ## Check if the number of matching VIIRS fire and geolocation files is the same
    for i in range(len(matching_VIIRS_fires)): ## Loop through the matching VIIRS fire and geolocation files
        unique_fire_id = get_unique_id_from_VIIRS_file(matching_VIIRS_fires[i]) ## Get the unique ID of the VIIRS fire file
        unique_geo_id = get_unique_id_from_VIIRS_file(matching_VIIRS_geolocations[i]) ## Get the unique ID of the VIIRS geolocation file
        con_list.append(unique_fire_id == unique_geo_id) ## Append the condition to the list
else: ## If the number of matching VIIRS fire and geolocation files is not the same
    print("The number of matching VIIRS fire and geolocation files is not the same. This is an error.")
    
if np.all(con_list): ## Check if all the conditions are true
    print("All the matching VIIRS fire and geolocation files have the same unique ID.")
else:
    print("Not all the matching VIIRS fire and geolocation files have the same unique ID.")
        

All the matching VIIRS fire and geolocation files have the same unique ID.


* Now we can create pandas dataframe for the files locations

In [21]:
matching_VIIRS_df = pd.DataFrame({"VIIRS_fire_files": matching_VIIRS_fires, "VIIRS_geolocation_files": matching_VIIRS_geolocations}) ## Create a dataframe of the matching VIIRS fire and geolocation files
matching_VIIRS_df[:5]

,VIIRS_fire_files,VIIRS_geolocation_files
0,F:\VIIRS_data\VIIRS_fire\VNP14IMG.A2022182.090...,F:\VIIRS_data\VIIRS_GEO\VNP03IMG.A2022182.0906...
1,F:\VIIRS_data\VIIRS_fire\VNP14IMG.A2022182.091...,F:\VIIRS_data\VIIRS_GEO\VNP03IMG.A2022182.0912...
2,F:\VIIRS_data\VIIRS_fire\VNP14IMG.A2022182.104...,F:\VIIRS_data\VIIRS_GEO\VNP03IMG.A2022182.1048...
3,F:\VIIRS_data\VIIRS_fire\VNP14IMG.A2022182.105...,F:\VIIRS_data\VIIRS_GEO\VNP03IMG.A2022182.1054...
4,F:\VIIRS_data\VIIRS_fire\VNP14IMG.A2022182.185...,F:\VIIRS_data\VIIRS_GEO\VNP03IMG.A2022182.1854...


* Now we will generate VIIRS points